# Pandas の groupby の使い方
https://qiita.com/propella/items/a9a32b878c77222630ae

In [1]:
import pandas as pd
import numpy as np

## DataFrame を作る

In [2]:
df = pd.DataFrame({
    'city': ['osaka', 'osaka', 'osaka', 'osaka', 'tokyo', 'tokyo', 'tokyo'],
    'food': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})
df

,city,food,price,quantity
0,osaka,apple,100,1
1,osaka,orange,200,2
2,osaka,banana,250,3
3,osaka,banana,300,4
4,tokyo,apple,150,5
5,tokyo,apple,200,6
6,tokyo,banana,400,7


In [3]:
df.columns # 列 label の取得

Index(['city', 'food', 'price', 'quantity'], dtype='object')

In [4]:
df.index # 行 label の取得

RangeIndex(start=0, stop=7, step=1)

## 簡単な groupby の使い方

余談終わり。groupby は、同じ値を持つデータをまとめて、それぞれの塊に対して共通の操作を行いたい時に使う。例えば一番簡単な使い方として、city ごとの price の平均を求めるには次のようにする。groupby で出来た GroupBy オブジェクトに対して、平均をとる mean メソッドを呼ぶと良い。

In [6]:
df.groupby('city').mean()

,price,quantity
city,,
osaka,212.5,2.5
tokyo,250.0,6.0


グループの指定に複数の label を指定する事も出来る。city と food の組み合わせで平均をとるには次のようにする。

In [7]:
df.groupby(['city', 'food']).mean()

price  quantity
city  food                   
osaka apple   100.0       1.0
      banana  275.0       3.5
      orange  200.0       2.0
tokyo apple   175.0       5.5
      banana  400.0       7.0

groupby を使うと、デフォルトでグループラベルが index になる。index にしたく無い場合は as_index=False を指定する。

In [8]:
df.groupby(['city', 'food'], as_index=False).mean()

,city,food,price,quantity
0,osaka,apple,100.0,1.0
1,osaka,banana,275.0,3.5
2,osaka,orange,200.0,2.0
3,tokyo,apple,175.0,5.5
4,tokyo,banana,400.0,7.0


## GroupBy オブジェクトの性質

デバッグ以外で使うところは無いかも知れないが、groupby によって作られた GroupBy オブジェクトの性質を調べるプロパティが幾つかある。まず、groupby によってどのように DataFrame が分割されたかを知るには groups を使う。{ 列 label: [行 label, 行 label, ...], ... } のような形で、どのグループにどの列が入ったか分かる。

In [9]:
df.groupby('city').groups

{'osaka': Int64Index([0, 1, 2, 3], dtype='int64'),
 'tokyo': Int64Index([4, 5, 6], dtype='int64')}

あるグループにどのようなデータが入ったかを知るには get_group を使う。

In [10]:
df.groupby('city').get_group('osaka')

,city,food,price,quantity
0,osaka,apple,100,1
1,osaka,orange,200,2
2,osaka,banana,250,3
3,osaka,banana,300,4


In [11]:
df.groupby('city').get_group('tokyo')

,city,food,price,quantity
4,tokyo,apple,150,5
5,tokyo,apple,200,6
6,tokyo,banana,400,7


各グループのサイズは size で取得出来る。

In [12]:
df.groupby('city').size()

city
osaka    4
tokyo    3
dtype: int64

size の結果は Series という一次元列を表すオブジェクトが返る。Series を使うと、osaka グループのサイズは添字を使って取得出来る。

In [13]:
df.groupby('city').size()['osaka']

4

## さまざまな Aggregation

GroupBy.mean() のように、グループごとに値を求めて表を作るような操作を Aggregation と呼ぶ。このように GroupBy オブジェクトには Aggregation に使う関数が幾つか定義されているが、これらは agg() を使っても実装出来る。

In [14]:
df.groupby('city').agg(np.mean)

"""
df.groupby('city').mean()
"""

,price,quantity
city,,
osaka,212.5,2.5
tokyo,250.0,6.0
